In [1]:
import json
import pandas as pd
from bs4 import BeautifulSoup 
import requests
from __future__ import print_function
from scipy.spatial import distance
import numpy as np

# 크롤링으로 여배우의 모든 사진을 다운받자

#### 하려 했으나 아래 주소는 따왔지만 다운이 안돼 image downloader라는 어플이용

In [6]:
base_url = "http://hentaku.net/list.php?sort=&page="

In [7]:
for i in range(1):
    page = i 
    new_url = base_url+str(page)
    data = requests.get(new_url)
    response = BeautifulSoup(data.content, "html.parser")
    response1 = response.select("div.list a img")
    for rs2 in response1:
        print(rs2['src'])



http://pics.dmm.com/mono/actjpgs/medium/aiuti_siori.jpg
http://pics.dmm.com/mono/actjpgs/medium/aika3.jpg
http://pics.dmm.com/mono/actjpgs/medium/aika_saya.jpg
http://pics.dmm.com/mono/actjpgs/medium/aikarin.jpg
http://pics.dmm.com/mono/actjpgs/medium/aikawa_seira.jpg
http://pics.dmm.com/mono/actjpgs/medium/aikawa_nana.jpg
http://pics.dmm.com/mono/actjpgs/medium/aikawa_megumi.jpg
http://pics.dmm.com/mono/actjpgs/medium/aikawa_yui3.jpg
http://pics.dmm.com/mono/actjpgs/medium/aikawa_rin.jpg
http://pics.dmm.com/mono/actjpgs/medium/aiku_yuu.jpg
http://pics.dmm.com/mono/actjpgs/medium/aisaka_haruna.jpg
http://pics.dmm.com/mono/actjpgs/medium/aisaki_kotone.jpg
http://pics.dmm.com/mono/actjpgs/medium/sakurai_ria.jpg
http://pics.dmm.com/mono/actjpgs/medium/aizawa_arisa.jpg
http://pics.dmm.com/mono/actjpgs/medium/aizawa_karin.jpg
http://pics.dmm.com/mono/actjpgs/medium/aizawa_hikaru.jpg
http://pics.dmm.com/mono/actjpgs/medium/aizawa_rina.jpg
http://pics.dmm.com/mono/actjpgs/medium/aizawa_ren.jp

# 여배우들의 얼굴을 벡터화시켜 저장해보자
## Faceplus API를 이용

In [41]:
def face_detect(imgurl):
    url = 'http://apius.faceplusplus.com/v2/detection/detect'
    headers = { 'User-Agent' : 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_5) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/45.0.2454.101 Safari/537.36',
               'Origin':'http://www.faceplusplus.com', 
               'Accept-Language':'ko-KR,ko;q=0.8,en-US;q=0.6,en;q=0.4', 
               'Content-type':'application/x-www-form-urlencoded', 
               'Accept':'*/*', 
               'Referer':'http://www.faceplusplus.com/demo-landmark/', 
               'Connection':'keep-alive'
              }
    values = {'attribute' : 'none',
              'url' : imgurl,
              'api_key' : 'e6f2c06ac2b93b1ae6aad80b90d49eb0',
              'api_secret' : '8o_amgigF4dV4EHZ2OOVEejDeXq4qFab'}
  
    response = requests.post(url,params=values,headers=headers)
    response_data = response.text
    return json.loads(response_data.decode('utf-8'))

In [42]:
## face landmark api call by face_id
def face_landmark(faceid):
    url = 'http://apius.faceplusplus.com/v2/detection/landmark'
    headers = { 'User-Agent' : 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_5) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/45.0.2454.101 Safari/537.36',
               'Origin':'http://www.faceplusplus.com', 
               'Accept-Language':'ko-KR,ko;q=0.8,en-US;q=0.6,en;q=0.4', 
               'Content-type':'application/x-www-form-urlencoded', 
               'Accept':'*/*', 
               'Referer':'http://www.faceplusplus.com/demo-landmark/', 
               'Connection':'keep-alive'
              }
    values = {'face_id' : faceid,
              'type' : '83p',
              'api_key' : 'e6f2c06ac2b93b1ae6aad80b90d49eb0',
              'api_secret' : '8o_amgigF4dV4EHZ2OOVEejDeXq4qFab'}
    
    response = requests.post(url,params=values,headers=headers)
    response_data = response.text
    return json.loads(response_data.decode('utf-8'))

def  extract_face_landmark( imgurl, face_index=0 ) :
    fd = face_detect(imgurl)
    face_id = fd['face'][face_index]['face_id']
    return face_landmark(face_id)

In [43]:
import json
def extract_face_feature_raw(facial_info):
    keypoint = {}
    
    ## json string to object
    info = facial_info
    if type(facial_info) == str:
        info = json.loads(facial_info)
    
    facial_pos = info['result'][0]['landmark']
    dx = facial_pos['contour_left1']['x']
    dy = facial_pos['contour_left1']['y']
    ## calculate key point
    for (k,v) in facial_pos.items(): ## iteritems() for python2.7
        keypoint[k+'_x'] = v['x']
        keypoint[k+'_y'] = v['y']
        
    return keypoint

keypoint = extract_face_landmark('http://historie.kr/img/face/goodvc_01.jpg')
extract_face_feature_raw(keypoint)

{u'contour_chin_x': 47.067083,
 u'contour_chin_y': 39.401944,
 u'contour_left1_x': 39.458125,
 u'contour_left1_y': 24.21875,
 u'contour_left2_x': 39.194583,
 u'contour_left2_y': 26.481556,
 u'contour_left3_x': 39.434583,
 u'contour_left3_y': 28.661111,
 u'contour_left4_x': 39.767917,
 u'contour_left4_y': 30.751111,
 u'contour_left5_x': 40.352083,
 u'contour_left5_y': 32.863889,
 u'contour_left6_x': 41.308958,
 u'contour_left6_y': 34.540833,
 u'contour_left7_x': 42.38625,
 u'contour_left7_y': 36.264167,
 u'contour_left8_x': 43.623333,
 u'contour_left8_y': 37.763611,
 u'contour_left9_x': 45.142083,
 u'contour_left9_y': 39.002222,
 u'contour_right1_x': 54.243958,
 u'contour_right1_y': 22.742722,
 u'contour_right2_x': 54.4525,
 u'contour_right2_y': 24.960806,
 u'contour_right3_x': 54.390833,
 u'contour_right3_y': 27.675639,
 u'contour_right4_x': 54.262917,
 u'contour_right4_y': 30.14,
 u'contour_right5_x': 53.841042,
 u'contour_right5_y': 32.415833,
 u'contour_right6_x': 53.051875,
 u'cont

In [44]:
def get_feature_vector( keypoint ):
    ff = extract_face_feature_raw(keypoint)
    return ff

# 아래에 비교하고자 하는 사진의 URL을 넣는다

In [45]:
img_names = """
http://dimg.donga.com/wps/NEWS/IMAGE/2013/11/25/59126402.3.jpg
http://spnimage.edaily.co.kr/images/photo/files/NP/S/2013/08/PS13083000034.jpg
http://kbau.co.kr/board/data/file/korean_3_2_3/1890292918_99134a95_4ECBD94EBAFB8EB9494EC96B8.PDF%2B-%2BAdobe%2BAcrobat%2BPro%2BExtended.jpg
http://img.mediapen.com/news/201412/58040_52411_855.jpg
http://img.tvreport.co.kr/images/20110813/20110813_1313198957_95780500_1.jpg
http://medicalworldnews.co.kr/data/news_image/1409/bea402bbde129801b7a00b1613243c56_7MzBsaTCyVc5j2ba9M.jpg
http://file2.nocutnews.co.kr/newsroom/image/2015/01/16/20150116010706614115.jpg
https://s-media-cache-ak0.pinimg.com/236x/f7/8a/4d/f78a4d57c61ad7533e292c8ea24066e0.jpg
"""

In [46]:
image_data = {  }


for link in img_names.split():
    name = (link.split('/')[-1].split('.')[0]).replace('','')
    image_data[name]= link

In [103]:
for n,u in image_data.items():
    print (n,u)

1890292918_99134a95_4ECBD94EBAFB8EB9494EC96B8 http://kbau.co.kr/board/data/file/korean_3_2_3/1890292918_99134a95_4ECBD94EBAFB8EB9494EC96B8.PDF%2B-%2BAdobe%2BAcrobat%2BPro%2BExtended.jpg
59126402 http://dimg.donga.com/wps/NEWS/IMAGE/2013/11/25/59126402.3.jpg
20150116010706614115 http://file2.nocutnews.co.kr/newsroom/image/2015/01/16/20150116010706614115.jpg
58040_52411_855 http://img.mediapen.com/news/201412/58040_52411_855.jpg
PS13083000034 http://spnimage.edaily.co.kr/images/photo/files/NP/S/2013/08/PS13083000034.jpg
f78a4d57c61ad7533e292c8ea24066e0 https://s-media-cache-ak0.pinimg.com/236x/f7/8a/4d/f78a4d57c61ad7533e292c8ea24066e0.jpg
20110813_1313198957_95780500_1 http://img.tvreport.co.kr/images/20110813/20110813_1313198957_95780500_1.jpg
bea402bbde129801b7a00b1613243c56_7MzBsaTCyVc5j2ba9M http://medicalworldnews.co.kr/data/news_image/1409/bea402bbde129801b7a00b1613243c56_7MzBsaTCyVc5j2ba9M.jpg


In [48]:
landmark_dict = {}
for name, img_url in image_data.items(): 
    try:
        keypoint = extract_face_landmark(img_url)
        landmark_dict[name] = get_feature_vector(keypoint)
    except:
        print ('extract fail : {0}'.format(name))

## 추출한 벡터를 DataFrame으로 바꿔준다

In [49]:
features_ds = pd.DataFrame.from_dict(landmark_dict, orient='index')
features_ds

,contour_right5_y,contour_right5_x,mouth_upper_lip_left_contour3_x,mouth_upper_lip_left_contour3_y,left_eye_upper_left_quarter_y,left_eye_upper_left_quarter_x,contour_right7_y,right_eyebrow_lower_right_quarter_y,left_eye_lower_right_quarter_x,left_eye_lower_right_quarter_y,...,left_eyebrow_upper_left_quarter_x,contour_chin_x,contour_chin_y,mouth_lower_lip_right_contour2_x,mouth_lower_lip_right_contour2_y,right_eye_bottom_y,right_eye_bottom_x,nose_contour_left2_y,left_eye_pupil_y,left_eye_pupil_x
1890292918_99134a95_4ECBD94EBAFB8EB9494EC96B8,62.256190,73.947368,46.795000,64.455714,40.368810,34.313947,71.420714,33.779048,39.535789,43.559286,...,30.885263,52.982105,79.576667,57.429474,66.010952,42.319048,62.115526,51.279286,41.707381,37.693684
20110813_1313198957_95780500_1,59.010922,58.226538,42.681154,62.854949,34.397270,33.607500,70.707850,25.243038,37.113269,37.358703,...,30.911731,46.406154,81.293857,49.225577,64.147440,34.998976,50.913269,46.069966,35.301365,35.562500
20150116010706614115,75.393989,50.004364,31.412364,74.212568,45.757104,25.197273,84.874317,43.319126,29.129455,49.771585,...,22.726909,34.420909,94.975137,41.308000,77.316667,51.937978,45.069636,60.828142,47.473224,28.055273
58040_52411_855,43.101667,60.943153,48.224588,43.796000,36.253167,42.911979,46.515833,33.938333,45.368363,37.004833,...,41.423527,51.229533,49.242000,54.426513,44.495167,36.504333,55.661089,39.685333,36.531000,44.011105
59126402,45.895500,62.279129,45.204447,46.095000,36.029833,38.931246,49.789000,33.348167,41.970712,37.270667,...,36.432030,48.540324,52.869833,52.041179,47.548500,36.889833,54.473803,40.599167,36.482833,41.024650
PS13083000034,47.574785,60.041865,48.421032,46.700000,30.817479,44.366270,54.090831,28.312722,46.545635,32.581089,...,42.973413,50.438095,58.977077,53.625992,49.481662,32.937249,55.994048,38.683095,31.380802,45.635516
bea402bbde129801b7a00b1613243c56_7MzBsaTCyVc5j2ba9M,48.335275,61.120727,50.431636,43.911003,29.793269,48.410182,54.153398,28.128932,49.894000,31.454272,...,47.462000,51.764182,57.047249,54.812545,47.788673,31.946602,56.936182,36.348544,30.355437,49.683636
f78a4d57c61ad7533e292c8ea24066e0,58.389150,79.199153,47.061017,60.049267,36.931085,33.236314,66.367742,32.548094,40.072881,39.978006,...,30.281398,51.935169,73.344575,60.725000,62.097361,40.582991,67.714831,47.828446,38.286217,37.808602


In [50]:
cd

C:\Users\KC


In [51]:
features_ds.to_csv('a.csv')

### 배우들의 facial point 를 저장한 뒤 아래와 같이 불러왔다

In [2]:
landmark_ds = pd.read_csv("landmark.csv", sep = ',', index_col = [0])

# 분석해보자

In [3]:
from sklearn.preprocessing import scale
from sklearn.preprocessing import normalize
from scipy.spatial import distance

In [15]:
ds_scale = pd.DataFrame(scale(landmark_ds), index = landmark_ds.index, columns = landmark_ds.columns)
ds_nor = pd.DataFrame(normalize(landmark_ds), index = landmark_ds.index, columns = landmark_ds.columns)
ds_scal_nor = pd.DataFrame(normalize(ds_scale), index = ds_scale.index, columns = ds_scale.columns)
ds_nor_scal = pd.DataFrame(scale(ds_nor), index = ds_nor.index, columns = ds_nor.columns)

In [10]:
def most_similar(name, ds, distance_func=distance.correlation):
    v1 = ds.loc[name]
    ret = []
    for name, row in ds.iterrows():
        ret.append( [name, round(distance_func(v1,row),5)] )
    return pd.DataFrame(ret,columns = ['name','distance']).sort(['distance'], ascending=True)

In [12]:
most_similar('azuma_rin', landmark_ds,distance.cosine)

C:\Anaconda3\lib\site-packages\ipykernel\__main__.py:6: FutureWarning: sort(columns=....) is deprecated, use sort_values(by=.....)


,name,distance
257,azuma_rin,-0.00000
703,katou_makoto,0.00013
326,harukawa_kasumi,0.00023
1694,sinzyou_miyuki,0.00025
1392,nisiyama_mari,0.00033
1027,megumi_sawa,0.00037
936,kuroki_kotone,0.00038
693,katahira_akane,0.00039
1022,maya_yuki,0.00040
1680,sinoda_ayumi,0.00040
